In [17]:
!pip install -r ../requirements.txt
!pip install sklearn
!pip install tensorflow addons

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
    ERROR: Command errored o

In [23]:
import tensorflow as tf
import pandas as pd
from sklearn.metrics import classification_report
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline
from collections import Counter

MAX_LENGTH = 200# Define the maximum number of words to tokenize (DistilBERT can tokenize up to 512)

# Create the model

In [24]:
tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa",from_tf=True,)
model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa",)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


# Load and transform the data

In [25]:
datasets = pd.read_csv('../datasets/train_preprocessed.csv').dropna()
X_train,y_train = datasets["content"].astype("string"),datasets["sentiment"].astype("category").cat.codes

In [26]:
# Define function to encode text data in batches
def batch_encode(tokenizer, texts, batch_size=256, max_length=MAX_LENGTH):
    """""""""
    A function that encodes a batch of texts and returns the texts'
    corresponding encodings and attention masks that are ready to be fed 
    into a pre-trained transformer model.
    
    Input:
        - tokenizer:   Tokenizer object from the PreTrainedTokenizer Class
        - texts:       List of strings where each string represents a text
        - batch_size:  Integer controlling number of texts in a batch
        - max_length:  Integer controlling max number of words to tokenize in a given text
    Output:
        - input_ids:       sequence of texts encoded as a tf.Tensor object
        - attention_mask:  the texts' attention mask encoded as a tf.Tensor object
    """""""""
    
    input_ids = []
    attention_mask = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             max_length=max_length,
                                             padding='max_length', #implements dynamic padding
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_token_type_ids=False
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])
    
    
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)
    
    
# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train.tolist())

# Create and compile the model

In [27]:
input_ids_layer = tf.keras.layers.Input(shape=(200,), 
                                        name='input_ids', 
                                        dtype='int32')
input_attention_layer = tf.keras.layers.Input(shape=(200,), 
                                              name='input_attention', 
                                              dtype='int32')
last_hidden_state = model.roberta([input_ids_layer, input_attention_layer])[0]

one_more = tf.keras.layers.AveragePooling1D(1,200)(last_hidden_state)
pooling = tf.keras.layers.Reshape((768,))(one_more)
output = tf.keras.layers.Dense(13,activation='softmax')(pooling)
roberta = tf.keras.Model([input_ids_layer, input_attention_layer], output)

In [29]:
params = {'optimizer':tf.keras.optimizers.Adam(learning_rate=1e-4),
          'loss':tf.keras.losses.CategoricalCrossentropy(),
          'metrics':["accuracy"],# tf.keras.metrics.top_k_categorical_accuracy(k=3)
}
roberta.compile(**params)

In [30]:
roberta.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 200)]        0           []                               
                                                                                                  
 input_attention (InputLayer)   [(None, 200)]        0           []                               
                                                                                                  
 roberta (TFRobertaMainLayer)   TFBaseModelOutputWi  124055040   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'input_attention[0][0]']        
                                tentions(last_hidde                                               
                                n_state=(None, 200,                                         

In [31]:
counts = Counter(y_train)
weights = {i:1/j for i,j in counts.items()}

In [34]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath="./esign_checkpoint/",save_best_only=True,save_weights_only=False),
    tf.keras.callbacks.EarlyStopping(patience=2,)
]

roberta.fit(x=[X_train_ids,X_train_attention],
            y=pd.get_dummies(y_train),
            batch_size=32,
            epochs = 8,
            workers = -1,
            validation_split=0.2,
            callbacks = callbacks,
            class_weight = weights,
            steps_per_epoch = 1
           )

Epoch 1/8
1/1 [==============================] - ETA: 0s - loss: 0.0013 - accuracy: 0.1250

KeyboardInterrupt: 

In [20]:
model.save_weights('./saved_weights.h5')

In [3]:
from sklearn.metrics import classification_report
import numpy as np

y_pred = np.argmax(model.predict(X_train),axis = 1)
print(classification_report(y_train, y_pred))

ModuleNotFoundError: No module named 'sklearn'